# __Creature Wars – Statistics__

Let's justify some of the numbers in the game, 
and investigate average game outcomes.

We will do this for the base game, ignoring the unique effects of Witches.

| Tribe | Base Str |
|-------|------------|
| Serpent | 0 |
| Wolf | 2 |
| Skeleton | 3 |
| Pirate | 4 |
| Giant | 5 |
| Vampire | 7 |
| Dragon | 9 |

- __Default levels__ *1-6*.

- Possible __Power Type Advantage__ for *+3 power*.

Remember, `power = base_str + level (+3 if type_advantage)`

---

First, let's examine the __min/max__ creature powers.

__Min.__

- Lvl.1 Serpent with _no_ Type Advantage &rightarrow; 1

__Max.__

- Lvl.6 Dragon with Type Advantage &rightarrow; 9 + 6 + 3 = 18

In [89]:
# Now, let's view every possible ordinary power value
power_vals = set()
base_strengths = (0, 2, 3, 4, 5, 7, 9)
levels = tuple(range(1,7))

for i in base_strengths:
	for j in levels:
		for k in (0, 3): # type advantage
			power_vals.add(i + j + k)

print(', '.join(str(val) for val in sorted(power_vals)))

missing = set(range(1,19)) - power_vals
if not missing:
	print('Every int from 1..18')
else:
	print('Missing:', missing)

1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18
Every int from 1..18


In [90]:
# Average power of a creature
avg_creature_power = sum(power_vals) / len(power_vals)
print(avg_creature_power)

9.5


In [91]:
# Average base strength of a creature
avg_base_str = sum(base_strengths) / len(base_strengths)
print(avg_base_str)

4.285714285714286


## Game Lengths

Remember, the last possible turn is marked once the final card is drawn. A final duel will take place, then the game concludes.

This is only true if an Empire has not been defeated during the previous turns, in which case the game concludes immediately.

Therefore we need to figure out 2 things:

#### 1. At which turn will the final card be drawn if no player has lost yet?

- How many cards will end up in each player's hand at the end of the game?

#### 2. The min, max, and average damage that can be dealt before the deck is empty.

- With this information, we can choose a starting HP for each player such that the game finishes after $N$ turns on average.

## Hand Size

The deck contains 52 cards:

- 42 creatures (leveled 1-6 from each Tribe)

- 10 random Witch cards (each has a *base str* of 3)

The game starts with 5 cards in each player's hand, then the first duel commences.

At the start of each turn afterwards, each player draws 1 card.

Each player also plays 1 card each duel.

Consequently, each player's hand will be at a constant size of 4 at the end of each turn.

$\therefore$ The game will end with __8 cards unplayed__.

&emsp;&emsp;Or, after $(52-8) \div 2 = 22$ turns.

---

However, 3 game mechanics (apart from Witch effects) may affect this!

- __Tribal Clashes__: a player draws an additional card when they win a duel where both creatures share the same Tribe.

- __Bizzare Victory__: a player gets a random Witch card when they win a duel where their creature was at a Power Type disadvantage.

- __Witches__: after a Witch card is played…

	1. against a __creature__ and __wins/ties__, the winner draws a card and the loser gets their creature back (both players gain 1 card)
	
	1. against a __creature__ and __loses__, the opponent gets their creature back (one player gains 1 card)

	1. against __another Witch__, both cards discarded as normal

We must calculate the probability that each event will occur.

## Tribal Clashes

There are $\dbinom{52}{2} = \dfrac{52 \times 51}{2} = 1326$ possible card-vs-card matchups.

A Tribal Clash only occurs when __both__ cards are creatures are from the __same tribe__.

Each tribe has 6 creatures, so same-tribe creature pairs per tribe: $\dbinom{6}{2} = \dfrac{6 \times 5}{2} = 15$

With 7 tribes, there are $7 \times 15 = 105$ pairs that would result in a Tribal Clash. 

__Probability of a Tribal Clash__: $\frac{105}{1326} = 7.92\%$

## Bizzare Victories

Each creature has a $1/4$ chance of being each of the 4 Power Types.

A Bizarre Victory only occurs when __both__ cards are creatures, one creature has a type advantage, and it loses.

We will assume a $1/5$ chance that the disadvantaged creature wins the duel.

There are $4 \times 4 = 16$ unique type-vs-type matchups.

Each Power Type is effective against only 1 other. Thus, there are $4$ type-advantage matchups.

We have:

- $1/4$ creature-vs-creature matchups result in a type advantage

- $1/20$ creature-vs-creature matchups result in a Bizarre Victory

There are $\dbinom{42}{2} = \dfrac{42 \times 41}{2} = 861$ possible creature-vs-creature matchups.

$\therefore \dfrac{861}{20} = 43.05$ creature-vs-creature matchups result in Bizarre Victories.

__Probability of Bizarre Victory__: $\dfrac{861 \div 20}{1326} = 3.25\%$

In [92]:
'''
Justification: Disadvantaged creature will win 1/5 of the time
'''

# How many base_str_a win against (base_str_b + 3)
disadvantage_wins = 0
total_pairs = 0
for x in base_strengths: # creature with disadvantage
	for y in base_strengths: # creature with +3 advantage
		if x > y + 3:
			disadvantage_wins += 1
		total_pairs += 1

print(f'{disadvantage_wins}/{total_pairs}')
print(f'{disadvantage_wins / total_pairs:.2%}')

10/49
20.41%


## Witch Cards

There are $\dbinom{10}{2} = 45$ unique witch-vs-witch matchups. 

And $10 \times 42 = 420$ witch-vs-creature matchups.

Witches have an approximately $56.88\%$ chance to successfully flee a duel against creatures (win/draw).

Therefore, we have:

1. $420 \times 56.88\% \approx 239$ witch-vs-creature matchups where the witch flees.

	__Probability__: $\frac{239}{1326} = 18.02\%$ &emsp;(of all card-vs-card matchups)

2. $420 - 239 = 181$ witch-vs-creature matchups where the witch dies.

	__Probability__: $\frac{181}{1326} = 13.65\%$

3. $45$ witch-vs-witch matchups

	__Probability__: $\frac{45}{1326} = 3.39\%$

In [93]:
'''
Justification: Witch flee rate (win/tie) against creatures
'''

# Witches have a constant base str of 3
# They can be level 1-9
witch_base_str = 3

witch_powers = [witch_base_str + i for i in range(1,10)]
creature_powers = [base + lvl for base in base_strengths for lvl in levels]

witch_wins = [0] * 9 # track for each possible level (counts ties)
total_pairs = 0

for c in creature_powers:
	for i in range(1,10): # witch level
		witch_power = witch_base_str + i
		if witch_power >= c:
			witch_wins[i-1] += 1
	total_pairs += 1

for i in range(9):
	print(f'Lvl.{i+1} flee rate: {witch_wins[i]}/{total_pairs} -> {witch_wins[i] / total_pairs:.2%}')

avg_witch_winrate = sum(witch_wins)/len(witch_wins)/total_pairs
print(f'\nAverage flee rate: {avg_witch_winrate:%}')

Lvl.1 flee rate: 7/42 -> 16.67%
Lvl.2 flee rate: 11/42 -> 26.19%
Lvl.3 flee rate: 16/42 -> 38.10%
Lvl.4 flee rate: 20/42 -> 47.62%
Lvl.5 flee rate: 25/42 -> 59.52%
Lvl.6 flee rate: 29/42 -> 69.05%
Lvl.7 flee rate: 33/42 -> 78.57%
Lvl.8 flee rate: 36/42 -> 85.71%
Lvl.9 flee rate: 38/42 -> 90.48%

Average flee rate: 56.878307%


## Hand Size (continued)

Each duel, we have the probabilities when playing 2 random cards from the deck:

- __Tribal Clashes__ (7.92%) — $t$

- __Bizarre Victories__ (3.25%) — $b$

- __Witch Flees__ (18.02%) — $f$

- __Witch Dies__ (13.65%) — $d$

In each case, except for a witch fleeing, 1 player draws or keeps their duel card in hand.  
This results in a card advantage of +1 for a single player.

When a witch flees, both players get to keep an extra card in hand.

Let's make an equation that tells us how many extra cards will be acquired by _either_ player,  
as a function of the number of turns passed $n$.

$$f(n) = (t + b + 2f + d)n$$

Or, the total hand size of both players after $n$ turns:

$$h(n) = (t + b + 2f + d)n + 10 - 2n + 2(n-1)$$

- Each player starts with 5 cards (10 total)

- 2 cards are played each turn (1 from each player)

- 2 cards are drawn every turn except the first

$$h(n) = (t + b + 2f + d)n + 8$$

## Game Lengths (1) – Final Draw

We also know that the cards held in hand, $h(n)$,  
plus the cards played, $2n$,  
plus the cards left in the deck, $u$, 
must equal 52.

Therefore, we have:

$$\begin{align*}
	52 &= h(n) + 2n + u \\
	52 &= (t + b + 2f + d)n + 8 + 2n + u \\
	52 &= (t + b + 2f + d + 2)n + 8 + u
\end{align*}$$

We want to figure out at which turn the deck will be empty, $u=0$

$$\begin{align*}
	52 &= (t + b + 2f + d + 2)n + 8 + 0 \\
	44 &= (t + b + 2f + d + 2)n \\
	44 &= [0.0792 + 0.0325 + 2(0.1802) + 0.1365 + 2]n \tag{substitute probabilities} \\
	44 &= (2.6086)n \\
	\therefore n &= \frac{44}{2.6086} \\ \\
	&= \boxed{16.87}
\end{align*}$$

At such turn, we can determine the total cards in hand:

$$h(16.87) = 18.27 \tag{cards unplayed}$$

If we divide these cards equally by both players:

$$\dfrac{h(16.87)}{2} = \boxed{9.13}$$

Therefore, if no player is defeated via damage, the game will end after __16.87 turns__ on average,  
with about __9.13 cards__ left in each player's hand.

We have solved the first problem!

Next, we will look at the min, max, and average damage that can be dealt before the deck is empty.

## Damage Dealt

Let's assume the game plays on until all 52 cards are drawn.

We know this will happen after ~17 turns on average.

This means 17 duels will take place, and 34 cards will be played.

However, for each duel only the losing card deals damage.

__Min. damage__

- We choose the 17 cards with the lowest base strength:

	- 6 Serpents — deal 0 each
	- 6 Wolves — deal 2 each
	- 5 Skeletons — deal 3 each

	$6(0) + 6(2) + 5(3) = 12 + 15 = \boxed{27}$ &nbsp; total damage

__Max. damage__

- We choose the 17 cards with the highest base strength:

	- 6 Dragons — deal 9 each
	- 6 Vampires — deal 7 each
	- 5 Giants — deal 5 each

	$6(9) + 6(7) + 5(5) = 54 + 42 + 25 = \boxed{121}$ &nbsp; total damage

__Avg. damage__

- The average base strength of a card is $4.04$

	$17(4.04) = \boxed{68.68}$ &nbsp; total damage

- This assumes every card has the same chance of losing a duel, which isn't true.  

	- Cards with a higher base strength are actually less likely  
	to lose a duel because they have more __Power__.

	- This means lower base strengths are more likely to have their damage dealt than higher base strengths.

	- So we can assume 68.68 is an __upper bound__ for the true average damage dealt.  
		This means that 70 is also an upper bound.

<br>

If the damage is evenly split between both players, we have:

$\dfrac{70}{2} = 35$ damage dealt to each player by turn 17, on average.

Therefore, if we set each player's health to 35, we would expect the game to end  
on turn $17-1=16$ (on average) because only one player needs to lose all their health.

This means roughly half the games would finish before turn 17. Or slightly _less_, since 70 damage was an upper bound.

- If less than 70 damage was dealt, and each player's health started at 35, it would take _longer_ to end the game.

In [94]:
'''
Justification: Average base damage of a card
'''

# First, tally creatures
all_base_strengths = sum(i for i in base_strengths for _ in range(6))

# Next, tally Witch cards
all_base_strengths += witch_base_str * 10

# Average
avg_card_base_str = all_base_strengths / 52
print(avg_card_base_str)

4.038461538461538


## Game Lengths (2) – Health Totals

Suppose one player has a duel win rate $r \in [0,1]$.

The other player then has win rate $s = 1 - r$.

- One player's win rate is the other's loss rate.

- The player who loses a __greater percentage of duels__ will lose the game on average.

- The greater loss rate between the two players is:

$$f(r, s) = \text{max}(r, s)$$

Since $s = 1-r$, the function becomes:

$$f(r) = \text{max}(r, 1-r)$$

#### Range of the Loss Rate Function:

We rewrite $f$ as piecewise:

$$f(r) = \begin{cases}
	1-r & \text{if } r \leq 1-r \\
	r  & \text{if } 1-r \lt r
\end{cases}$$

Let's evaluate each inequality:

1. $r \leq 1-r \implies 2r \leq 1 \implies r \leq 0.5$

1. $1-r \lt r \implies 1 \lt 2r \implies 0.5 \lt r$

Since $r \in [0,1]$, we have:

1. $0 \le r \leq 0.5$

1. $0.5 \lt r \le 1$

Our piecewise function becomes:

$$f(r) = \begin{cases}
	1-r & \text{if } 0 \le r \leq 0.5 \\
	r  & \text{if } 0.5 \lt r \le 1
\end{cases}$$

If $0 \le r \leq 0.5$,

- $f(r) = 1-r \rightarrow [0.5, 1]$

If $0.5 \lt r \le 1$,

- $f(r) = r \rightarrow (0.5, 1]$

Thus, the range of the greater loss rate is:

$$[0.5,1]$$

Meaning: the player who loses more duels always loses __at least 50%__ and __at most 100%__ of duels.

We can clearly see that the range of $f(r,s)$ is $[0.5, 1]$

This makes sense, as the loss rates between the players must sum to 1.

---

#### Determining Health Totals

Now, let's determine the health, $H$, of each player such that the game ends on turn $N$, on average.

We know the average damage each card deals is $x = 4.04$.

Let $u \in [0.5, 1]$ be the loss rate of the _losing player_.

- The losing player takes $ux$ damage per turn.

- After $N$ turns, they take $Nux$ damage.

We choose $H$ such that the losing player reaches 0 HP on turn $N$:

$$H - Nux = 0$$

Therefore,

$$N = \dfrac{H}{ux}$$

- As damage $x$ increases, game length $N$ decreases

- As loss rate $u$ increases, game length $N$ decreases

- As health total $H$ increases, game length $N$ increases

#### __Special Case__: $u=0.5$

This is the symmetric case where __both players__ have the same win rate.

That means:

- After $N$ turns, each player has taken the same damage: $N \cdot \frac{1}{2}x$

If we set $H = Nux$, then both players reach 0 HP simultaneously.  
But only _one_ player needs to die for the game to end.

- The turn before both players reach 0 HP, one player must be alive while the other is dead because only one player takes damage per turn.

Therefore,

$$N_\text{end} = N - 1 = \dfrac{H}{ux} - 1$$

In other words, set:

$$H = (N_\text{end} + 1)ux$$

- Then, they are both dead. But we only need one to die.

- In such case, we will round $N$ down to the nearest odd integer.

	- For if $N$ is odd, one player would have had to take more damage.

	- We round _down_ because $N$ would represent the turn they are both dead and we want one to be alive (to win).

## Show Health Totals

In [135]:
def show_health_totals(avg_damage, loss_rate):
	if loss_rate < 0.5 or loss_rate > 1:
		raise ValueError('loss_rate: [0.5, 1]')
	print('Health total to end the game by turn N (on average):')
	for n in range(1, 23):
		if loss_rate <= 0.55: # special case: (close to same win rate):
			hp = (n + 1) * avg_base_str * loss_rate
		else:
			hp = n * avg_damage * loss_rate
		s = f'- Turn {n}: {round(hp, 2)}'
		if n == 17:
			s += ' (the deck is usually fully drawn here)'
		elif n == 22:
			s += ' (the deck MUST be fully drawn here)'
		print(s)

In [142]:
show_health_totals(avg_card_base_str, 0.50) # 4.04, 50% loss rate

Health total to end the game by turn N (on average):
- Turn 1: 4.29
- Turn 2: 6.43
- Turn 3: 8.57
- Turn 4: 10.71
- Turn 5: 12.86
- Turn 6: 15.0
- Turn 7: 17.14
- Turn 8: 19.29
- Turn 9: 21.43
- Turn 10: 23.57
- Turn 11: 25.71
- Turn 12: 27.86
- Turn 13: 30.0
- Turn 14: 32.14
- Turn 15: 34.29
- Turn 16: 36.43
- Turn 17: 38.57 (the deck is usually fully drawn here)
- Turn 18: 40.71
- Turn 19: 42.86
- Turn 20: 45.0
- Turn 21: 47.14
- Turn 22: 49.29 (the deck MUST be fully drawn here)


In [148]:
show_health_totals(avg_card_base_str, 0.80) # 4.04, 80% loss rate

Health total to end the game by turn N (on average):
- Turn 1: 3.23
- Turn 2: 6.46
- Turn 3: 9.69
- Turn 4: 12.92
- Turn 5: 16.15
- Turn 6: 19.38
- Turn 7: 22.62
- Turn 8: 25.85
- Turn 9: 29.08
- Turn 10: 32.31
- Turn 11: 35.54
- Turn 12: 38.77
- Turn 13: 42.0
- Turn 14: 45.23
- Turn 15: 48.46
- Turn 16: 51.69
- Turn 17: 54.92 (the deck is usually fully drawn here)
- Turn 18: 58.15
- Turn 19: 61.38
- Turn 20: 64.62
- Turn 21: 67.85
- Turn 22: 71.08 (the deck MUST be fully drawn here)


## Heath Totals – Conclusion

Let us choose $H = 42$ so that:

- When $u=0.5$ the game finishes around __Turn 19__

	-  _After_ the deck is usually drawn (turn 17), and the game finishes by Mayhem.

	- In this case, the damage is evenly split between both players.
	
	- This suggests the game is very close and no player has a clear advantage.

- When $u=0.8$ the game finishes around __Turn 13__

	- _Before_ the deck is fully drawn, and one player has lost all their HP.

	- In this case, one player has won 80% of the duels.

	- The game still lasts several turns but does not go to Mayhem.

<br>

Remember, games are biased to last _longer_ than predicted,  
because cards with higher base strength are more likley to win their duel.

Also keep in mind that all of this has ignored the additional effects of Witch cards which change the rules of the game.

- Cards could be added/discarded from the deck to extend/terminate the game length.

## Rallies

A __Rally__ occurs when a player holds at least __4 creatures__ of the same Tribe or Power Type,
__or__ at least __4 Witch cards__.

We are going to figure out the probability of having the _opportunity_ to get various Rallies.

- This means you have drawn at least 4 matching cards.

- Note: You may play one of the matching cards before a Rally forms  
	(due to necessity or not knowing what cards you'd get).

---

__Deck Setup__

- Total deck: 52 cards

- 42 creatures

- 10 witches

Number of total unique hands of size $n$:

$$\text{Total hands} = \dbinom{52}{n}$$

Let's start with a hand of size $n=4$ and work to derive a generic formula.

### 1. Tribes

- There are $7$ tribes to choose from.

- Each tribe has $6$ creatures.

- We need $\ge 4$ creatures from the same tribe to form a Tribe Rally.

- With just $4$ cards, we have $7 \cdot \dbinom{6}{4} = 105$ possible Tribe Rally hands.

Now, given a hand of size $n$, the number of matching creatures must be at least 4, but can also be 5, 6, …, up to $n$ (hand size).

- Actually, the __maximum__ number of matches we can have is 6,  
	because there are only a maximum of 6 creatures in the deck with the same tribe.

We will create a function to calculate the number of hands with exactly $k$ matching creatures, where $4 \le k \le \text{min}(n, 6)$

Favorable hands of size $n$ with $k$ matches:

$$F_\text{tribe}(n, k) = 7 \cdot \dbinom{6}{k} \dbinom{52-6}{n-k}$$

"For each of the 7 tribes, there are 6 choose $k$ ways to select $k$ creatures from such tribe,  
and the other $(n-k)$ cards must come from a different tribe or no tribe (from the 46 remaining cards)."

Next, we can create a summation that sums all the favorable hands for all possible $k$ in our domain.

Favorable hands with _any_ number of matches (at least 4):

$$F_\text{tribe}(n) = 7 \cdot \sum_{k=4}^{\text{min}(n, 6)} \dbinom{6}{k} \dbinom{46}{n-k}$$

Probability of Tribe Rally:

$$P_\text{tribe}(n) = \dfrac{F_\text{tribe}(n)}{\binom{52}{n}}$$

### 2. Power Types

- Each creature has a $25\%$ chance of being any of the $4$ Power Types.

- Let there be $x_i$ creatures corresponding to Power Type $i$.

- We need $\ge 4$ creatures with the same Power Type to form a Rally.

- Therefore, with just $4$ cards we have $\sum_{i=1}^4 \dbinom{x_i}{4}$ possible Power Rally hands.

Let's assume an even spread of Power Types $(10, 10, 11, 11)$ in the deck.

In this case, $n=4$, we have $2 \dbinom{10}{4} + 2 \dbinom{11}{4} = 1080$ Power Rallies.

Like before, we will generalize to any $n$ for exactly $k$ matches of a Power Type with $x$ cards:

$$F_\text{power}(n, k, x) = \dbinom{x}{k} \dbinom{52-x}{n-k}$$

Next, we must allow for any number of matches $k$.

In this case, $k$ must still be at least 4, and at most the hand size $n$,  
but also has the limit that there are only $x$ cards of a single Power Type in the deck.

Therefore, $4 \le k \le \text{min}(n, x)$.

For a given type with $x$ cards in the deck, we have:

$$F_\text{power}(n, x) = \sum_{k=4}^{\text{min}(n,x)} \dbinom{x}{k} \dbinom{52-x}{n-k}$$

Finally, we must count matches from any Power Type as favorable.

Favorable hands:

$$F_\text{power}(n) = \sum_{i=1}^{4} \sum_{k=4}^{\text{min}(n,x_i)} \dbinom{x_i}{k} \dbinom{52-x_i}{n-k}$$

For our equally distributed case $S = \{10, 10, 11, 11\}$:

$$F_\text{power}(n) = \sum_{x \in S} \sum_{k=4}^{\text{min}(n,x)} \dbinom{x}{k} \dbinom{52-x}{n-k}$$

Probability of Power Rally:

$$P_\text{power}(n) = \dfrac{F_\text{power}(n)}{\binom{52}{n}}$$

### 3. Witches

- There are 10 Witch cards.

- We need at least 4 witches to form a Rally.

- With just $4$ cards, there are $\dbinom{10}{4} = 210$ possible Witch Rallies.

For $n$ cards with $k$ witches, we have:

$$F_\text{witch}(n, k) = \dbinom{10}{k} \dbinom{42}{n-k}$$

Of course, here $k$ is limited by the hand size $n$ as well as the number of Witch cards in the deck, $10$.

$\therefore 4 \le k \le \text{min}(n, 10)$

Favorable hands:

$$F_\text{witch}(n) = \sum_{k=4}^{\text{min}(n,10)} \dbinom{10}{k} \dbinom{42}{n-k}$$

Probability of Witch Rally:

$$P_\text{witch}(n) = \dfrac{F_\text{witch}(n)}{\binom{52}{n}}$$

### 4. Double Rally

- It is possible that a player gets a Rally where creatures share __both__ Tribe and Power Type.

- However, without knowing the __exact distribution__ of power types within each tribe,  
the intersection $P_\text{double} = P(\text{tribe} \cap \text{power})$ cannot be determined precisely.

- We can proceed by making some assumptions.

	1. __Best-case__ distribution (maximizes intersection)

	1. __Worst-case__ distribution (minimizes intersection)

	1. __Average-case__, evenly distributed

<br>

__Worst Case__ (no intersection):

If every tribe splits its 6 creatures across all 4 power types as evenly as possible, e.g.:

- 6 = 2 + 2 + 1 + 1

Then every (Tribe, Power) combination has a max size of 2.

In this case, it is __impossible__ to ever draw 4 matching.

$\therefore P_\text{double}(n) = 0$

<br>

__Best Case__ (complete intersection):

If every tribe has only a single power type for all its 6 creatures, then every Tribe Rally is gauranteed to form a Power Rally.

- __Ex.__ All Serpents are `Fire` types, and we find 4 Serpents. We have also found 4 `Fire` types!

- However, if we find 4 `Fire` types, we may have found 2 Serpents and 2 Wolves.

	- There are only 4 Power Types but 7 Tribes, so some tribes will share the same power type.

$\therefore P_\text{double}(n) = P_\text{tribe}(n) \lt P_\text{power}(n) $

<br>

__Average Case__:

Let us find how many decks on average contain at least 1 Double Rally.

For a single Tribe, we have 6 creatures.

Each creature independently has a $\frac{1}{4}$ chance to be any of the 4 Power Types.

There are $4^6 = 4096$ possible sequences representing type assignments for 6 creatures.

- __Ex.__ $\langle F,F,H,F,H,N,A \rangle$ is a single sequence with 1 Ancient, 3 Fire, 1 Nightmare, and 1 Holy.

- We will express the type counts as: $(1, 3, 1, 1)$.

- There are multiple sequences that have the same totals for each Power Type.

Assume the 4 Power Types are in an arbitrary but fixed order.

Let $x_i$ be the number of creatures belonging to Power Type $i$.

A Tribe cannot have a Double Rally if $x_i \le 3 \ \forall_i$

- In other words, the Tribe does not contain at least 4 creatures of a shared Power Type.

Let us count all sequences where no type exceeds 3.

__Partitions of 6 with Parts &le; 3__

| Counts <br>$(x_1, x_2, x_3, x_4)$ | Permutations <br><br>$\left(\dfrac{6!}{x_1! x_2! x_3! x_4!}\right)$ | # Choices for Power Types | Total Sequences |
|---|:-:|:-:|--:|
| (3,3,0,0) | $\frac{6!}{3!3!0!0!} = 20$ | $\dbinom{4}{2} = 6$ | $20 \times 6 = 120$ |
| (3,2,1,0) | $\frac{6!}{3!2!1!0!} = 60$ | $4! = 24$ | $60 \times 24 = 1440$ |
| (3,1,1,1) | $\frac{6!}{3!1!1!1!} = 120$ | $\dbinom{4}{1} = 4$ | $120 \times 4 = 480$ |
| (2,2,2,0) | $\frac{6!}{2!2!2!0!} = 90$ | $\dbinom{4}{3} = 4$ | $90 \times 4 = 360$ |
| (2,2,1,1) | $\frac{6!}{2!2!1!1!} = 180$ | $\dbinom{4}{2} = 6$ | $180 \times 6 = 1080$ |
| __Total__ | — | — | $3480$ |

- We multiply by the number of ways for selecting the Power Types because we have chose a fixed order of types (1, 2, 3, 4).

- The counts (3,3,0,0) then represent 3 creatures of type 1 and 2. However, they could have been from any $\dbinom{4}{2}$ of the Power Types.

Thus,

$$P_\text{none, single} = \dfrac{3480}{4096} = 0.849609 \approx 84.96\%$$

Power Type assignments to creatures are independent across Tribes (each of the 42 creatures chooses independently).

Therefore, probability the deck has __0% Double Rally chance__:

$$p = [P_\text{none, single}]^7 = \left(\dfrac{3480}{4096} \right)^7 = 0.319547 \approx \boxed{31.95\%}$$

Probability the deck contains __at least one Double Rally__:

$$1-p = 0.680453 \approx \boxed{68.05\%}$$

It will only be possible to get a Double Rally in about 68% of games!

---

Now, let's estimate the chance of getting a Double Rally after drawing $n$ cards on average, given the deck contains one.

We had, 

$P_\text{none, single} = \dfrac{3480}{4096}$

Therefore, the probability that a single Tribe will contain at least one Double Rally is:

$P_{1_+\text{, single}} = \dfrac{616}{4096} \approx 0.15$

If each tribe independently has a 15% chance to hold a Double Rally, then:

$7 \cdot (P_{1_+\text{, single}}) \approx 7(0.15) = 1.052734375$

Consequently, we can expect about 1 out of the 7 Tribes to hold a Double Rally.

We also know that 4 matches (Tribe, Power) are more likely than 5 or 6.

So, let's assume that on average games that have a Double Rally present will have only 4 matching (Tribe, Power).

Then, 

$$F_\text{double}(n) = \dbinom{48}{n-4}$$

Probability of Double Rally:

$$P_\text{double}(n) = \dfrac{F_\text{double}(n)}{\binom{52}{n}}$$

### 5. Any Rally

- The probability to get a Tribe, Power, or Witch Rally is the sum of their probabilities minus any intersections.

- A Witch Rally is independent from creature Rallies becauses witches are separate cards.

- We have already determined the probability of a Double Rally.

$$P_\text{total}(n) = P_\text{tribe} + P_\text{power} + P_\text{witch} - P_\text{double}$$

In [84]:
'''
Let's calculate the probabilities of getting
various Rallies, after drawing n cards.
'''

def factorial(n):
	if n == 0:
		return 1
	if n < 0:
		raise ValueError('factorial() undefined for n < 0')
	total = 1
	while n > 1:
		total *= n
		n -= 1
	return total

def permute(n, r):
	if not (n >= 0 and r >= 0):
		raise ValueError('permute() undefined for n, r < 0')
	if n < r:
		return 0
	return int(factorial(n) / factorial(n-r))

def choose(n, r):
	return int(permute(n,r) / factorial(r))

def p_tribe(n):
	if n < 4:
		return 0
	
	total = choose(52, n)
	fav = 0

	for k in range(4, min(n, 6) + 1):
		fav += 7 * choose(6, k) * choose(46, n - k)
	return fav / total

def p_power(n):
	if n < 4:
		return 0
	
	power_counts = [10, 10, 11, 11]
	total = choose(52, n)
	fav = 0

	for x in power_counts:
		for k in range(4, min(n, x) + 1):
			fav += choose(x, k) * choose(52 - x, n - k)
	return fav / total

def p_witch(n):
	if n < 4:
		return 0
	
	total = choose(52, n)
	fav = 0

	for k in range(4, min(n, 10) + 1):
		fav += choose(10, k) * choose(42, n - k)
	return fav / total
	
def p_double(n):
	if n < 4:
		return 0
	return choose(48, n-4) / choose(52, n)

def p_total(n):
	if n < 4:
		return 0
	return p_tribe(n) + p_power(n) + p_witch(n) - p_double(n)

## Show Rally Probabilities

In [87]:
# Rallies for Hand Size (n)
for i in range(4, 11):
	t = p_tribe(i)
	p = p_power(i)
	w = p_witch(i)
	d = p_double(i)
	total = p_total(i)
	print(f'After drawing n={i}')
	print(f'Tribe Rally: {t:.2%}')
	print(f'Power Rally: {p:.2%}')
	print(f'Witch Rally: {w:.2%}')
	print(f'Double Rally: {d:.2%}')
	print(f'Any Rally: {total:.2%}')
	print(f'Power / Tribe: {p / t:.2f}')
	print('----------')

After drawing n=4
Tribe Rally: 0.04%
Power Rally: 0.40%
Witch Rally: 0.08%
Double Rally: 0.00%
Any Rally: 0.51%
Power / Tribe: 10.29
----------
After drawing n=5
Tribe Rally: 0.19%
Power Rally: 1.77%
Witch Rally: 0.35%
Double Rally: 0.00%
Any Rally: 2.31%
Power / Tribe: 9.47
----------
After drawing n=6
Tribe Rally: 0.54%
Power Rally: 4.73%
Witch Rally: 0.94%
Double Rally: 0.01%
Any Rally: 6.21%
Power / Tribe: 8.71
----------
After drawing n=7
Tribe Rally: 1.22%
Power Rally: 9.80%
Witch Rally: 1.97%
Double Rally: 0.01%
Any Rally: 12.98%
Power / Tribe: 8.00
----------
After drawing n=8
Tribe Rally: 2.36%
Power Rally: 17.36%
Witch Rally: 3.53%
Double Rally: 0.03%
Any Rally: 23.23%
Power / Tribe: 7.35
----------
After drawing n=9
Tribe Rally: 4.10%
Power Rally: 27.65%
Witch Rally: 5.69%
Double Rally: 0.05%
Any Rally: 37.40%
Power / Tribe: 6.74
----------
After drawing n=10
Tribe Rally: 6.59%
Power Rally: 40.73%
Witch Rally: 8.48%
Double Rally: 0.08%
Any Rally: 55.71%
Power / Tribe: 6.18
-

#### Results

It seems that a Power Rally is much more likely than either a Tribe or Witch Rally (which have roughly the same probability).

This makes sense because there are only 10 Witch cards, 
and the 42 creatures are split over 7 Tribes compared to just 4 Power Types, making a match of types more common.

It also seems that as $n$ increases, the gap between Power Rallies and the other Rallies decreases.

If we hold cards in anticipation of a Rally, we have about a 50% chance to find one after 10 draws!

## __Summary of Findings__

For the base game (ignoring the extra effects of Witches).


| Title | Value | Topic |
|------|:-----:|:-----:|
| Deck Size | 52
| Creatures | 42
| Witch Cards | 10
| Lesser Witch Weight | 3/6
| Sacred Witch Weight | 2/6
| Sinful Witch Weight | 1/6
| __Card-vs-Card Matchup Chance__ | — | |
| Creature-vs-Creature | 64.93%
| Witch-vs-Creature (Witch flees) | 18.02% | [Witch Cards](#witch-cards)
| Witch-vs-Creature (Witch dies) | 13.65%
| Witch-vs-Witch | 3.39%
| Avg. Witch Flee Rate | 56.88%
| Tribal Clash Probability | 7.92% | [Tribal Clash](#tribal-clashes)
| Bizarre Victory Probability | 3.24% | [Bizarre Victory](#bizzare-victories)
| __All 52 Cards are Drawn__ | — | 
| Hand Size Function | $$h(n) = (t + b + 2f + d)n + 8$$ | [Hand Size](#hand-size-continued)
| Avg. Turn Deck will be Empty | 16.87 | [Final Draw](#game-lengths-1--final-draw)
| Avg. Cards in each Player's Hand | 9.13
| Min. Damage Dealt | 27 | [Damage](#damage-dealt)
| Max. Damage Dealt | 121
| Avg. Damage Dealt | 68.68
| Avg. Damage per Card | 4.04
| __Health Totals__ | — | |
| Health Equation | $$N = \dfrac{H}{ux}$$ | [Health Totals](#game-lengths-2--health-totals)
| Health Totals per Game Length | — | [Show Totals](#show-health-totals)
| __Rallies__ | — | |
| Total $n$-Card Hands | $\dbinom{52}{n}$
| Tribe Rally Hands | $$F_\text{tribe}(n) = 7 \cdot \sum_{k=4}^{\text{min}(n, 6)} \dbinom{6}{k} \dbinom{46}{n-k}$$ | [Tribe Rally](#1-tribes)
| Power Rally Hands | $$F_\text{power}(n) = \sum_{x \in S} \sum_{k=4}^{\text{min}(n,x)} \dbinom{x}{k} \dbinom{52-x}{n-k}$$ | [Power Rally](#2-power-types)
| Witch Rally Hands | $$F_\text{witch}(n) = \sum_{k=4}^{\text{min}(n,10)} \dbinom{10}{k} \dbinom{42}{n-k}$$ | [Witch Rally](#3-witches)
| Double Rally Hands | $$F_\text{double}(n) = \dbinom{48}{n-4}$$ | [Double Rally](#4-double-rally)
| Decks with No Double Rallies | 31.95%
| Decks with at least 1 Double Rally | 68.05%
| Rally Chance per Draws | — | [Show Rallies](#show-rally-probabilities)